In [1]:
%pip install torch -q 
%pip install transformers -q 
%pip install numpy -q 
%pip install langchain -q 
%pip install langchain_community -q 
%pip install langchain-chroma -q 
%pip install langchain_transformers -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Initialize HuggingFace LLM


In [2]:
# Install python-dotenv to load .env files
%pip install python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from langchain_community.llms import HuggingFaceHub

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv('HUGGINGFACE_API_TOKEN')

if not api_key:
    raise ValueError("HUGGINGFACE_API_TOKEN not found in .env file. Please check your .env file.")

# Initialize the HuggingFace llm
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    model_kwargs={"temperature": 0.1, "max_length": 500},
    huggingfacehub_api_token=api_key
)

print("✅ HuggingFace LLM initialized successfully!")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_43356\2339488867.py:15: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


✅ HuggingFace LLM initialized successfully!
